In [1]:
#Step 1: Import & Initialize Earth Engine
import ee
import geemap

# Authenticate (run once)
ee.Authenticate()

# Initialize
ee.Initialize()

# Create map
Map = geemap.Map()

ModuleNotFoundError: No module named 'ee'

In [ ]:
#Step 2: Load Study Area
studyArea = ee.FeatureCollection('projects/ee-geogalo-2025/assets/Borana')

Map.centerObject(studyArea, 10)
Map.addLayer(studyArea, {}, "Study Area")

In [ ]:
#Step 3: Define Date Range
startDate = '1990-01-01'
endDate = '1990-12-31'

In [ ]:
#Step 4: Scaling & Cloud Mask Functions
def applyScaleFactors(image):
    opticalBands = image.select('SR_B.*').multiply(0.0000275).add(-0.2)
    return image.addBands(opticalBands, None, True)


def maskClouds(image):
    qa = image.select('QA_PIXEL')
    
    cloudShadowBitMask = 1 << 3
    snowIceBitMask = 1 << 4
    cloudBitMask = 1 << 5
    
    mask = (qa.bitwiseAnd(cloudShadowBitMask).eq(0)
            .And(qa.bitwiseAnd(snowIceBitMask).eq(0))
            .And(qa.bitwiseAnd(cloudBitMask).eq(0)))
    
    return image.updateMask(mask)

In [ ]:
Step 5: Landsat Processing Function
def processLandsat(image):
    scaled = applyScaleFactors(image)
    masked = maskClouds(scaled)
    
    return (masked.select(
        ['SR_B1','SR_B2','SR_B3','SR_B4','SR_B5','SR_B7'],
        ['Blue','Green','Red','NIR','SWIR1_1610','SWIR2_2200']
    ).copyProperties(image, ['system:time_start']))

In [ ]:
#Step 6: Load Landsat 4 & 5
landsat5 = (ee.ImageCollection("LANDSAT/LT05/C02/T1_L2")
            .filterBounds(studyArea)
            .filterDate(startDate, endDate)
            .filter(ee.Filter.lt('CLOUD_COVER', 30))
            .map(processLandsat))

landsat4 = (ee.ImageCollection("LANDSAT/LT04/C02/T1_L2")
            .filterBounds(studyArea)
            .filterDate(startDate, endDate)
            .filter(ee.Filter.lt('CLOUD_COVER', 30))
            .map(processLandsat))

landsatCollection = landsat5.merge(landsat4)

medianComposite = landsatCollection.median().clip(studyArea)

Map.addLayer(medianComposite,
             {'bands':['SWIR1_1610','NIR','Green'],
              'min':0,'max':0.3},
             "1990 RGB Composite")

In [ ]:
#Step 7: Spectral Indices
ndvi = medianComposite.normalizedDifference(['NIR', 'Red']).rename('NDVI')

bsi = medianComposite.expression(
    '((SWIR1_1610 + Red) - (NIR + Blue)) / ((SWIR1_1610 + Red) + (NIR + Blue))',
    {
        'SWIR1_1610': medianComposite.select('SWIR1_1610'),
        'Red': medianComposite.select('Red'),
        'NIR': medianComposite.select('NIR'),
        'Blue': medianComposite.select('Blue')
    }).rename('BSI')

ndbi = medianComposite.normalizedDifference(['SWIR1_1610', 'NIR'])
ibi = ndbi.subtract(ndvi).divide(ndbi.add(ndvi)).rename('IBI')

mndwi = medianComposite.normalizedDifference(['Green','SWIR1_1610']).rename('MNDWI')

srtm = ee.Image('USGS/SRTMGL1_003').select('elevation').clip(studyArea).rename('DEM')
slope = ee.Terrain.slope(srtm).rename('Slope')

In [ ]:
#Step 8: Stack Classification Features
classificationFeatures = (medianComposite
    .select(['Blue','Green','Red','NIR','SWIR1_1610','SWIR2_2200'])
    .addBands(ndvi)
    .addBands(bsi)
    .addBands(ibi)
    .addBands(mndwi)
    .addBands(srtm)
    .addBands(slope)
    .toFloat())

print("Bands used for classification:")
print(classificationFeatures.bandNames().getInfo())

In [ ]:
#Step 9: Load Ground Truth
groundTruth = ee.FeatureCollection(
    'projects/ee-geogalo-2025/assets/GT_2013_dissolve_V1')

classProperty = 'ClassValue'

groundTruth = groundTruth.filter(
    ee.Filter.notNull([classProperty]))

In [ ]:
#Step 10: Sample Training Data
trainingData = classificationFeatures.sampleRegions(
    collection=groundTruth,
    properties=[classProperty],
    scale=30,
    tileScale=4
)

print("Training samples:", trainingData.size().getInfo())

In [ ]:
#Step 11: Train / Validation Split
splits = trainingData.randomColumn('random', 42)

trainingSet = splits.filter(ee.Filter.lt('random', 0.8))
validationSet = splits.filter(ee.Filter.gte('random', 0.8))

In [ ]:
#Step 12: Train Random Forest
inputFeatures = classificationFeatures.bandNames()

classifier = (ee.Classifier.smileRandomForest(
    numberOfTrees=500,
    seed=42)
    .train(
        features=trainingSet,
        classProperty=classProperty,
        inputProperties=inputFeatures))

In [ ]:
#Step 13: Calssification
classifiedImage = classificationFeatures.classify(classifier)
classifiedImage = classifiedImage.updateMask(classifiedImage.gte(1))

classVis = {
    'min':1,
    'max':8,
    'palette':[
        '#B4B4B4',
        '#FA0000',
        '#F096FF',
        '#006400',
        '#FFFF4C',
        '#FFBB22',
        '#0064C8',
        '#00CF75'
    ]
}

Map.addLayer(classifiedImage, classVis, "LULC Classification")
Map

In [ ]:
#Step 14: Accuracy Assessment
trainMatrix = classifier.confusionMatrix()
print("Training Accuracy:", trainMatrix.accuracy().getInfo())
print("Training Kappa:", trainMatrix.kappa().getInfo())

validated = validationSet.classify(classifier)
validationMatrix = validated.errorMatrix(classProperty, 'classification')

print("Validation Accuracy:", validationMatrix.accuracy().getInfo())
print("Validation Kappa:", validationMatrix.kappa().getInfo())

In [ ]:
#Step 15: Export to Drive
task = ee.batch.Export.image.toDrive(
    image=classifiedImage.int(),
    description='LULC_Classification_Borena_1990',
    folder='GEE_LULC_Classifications',
    fileNamePrefix='LULC_Classification_1990',
    region=studyArea.geometry().bounds(),
    scale=30,
    maxPixels=1e13,
    fileFormat='GeoTIFF'
)

task.start()

print("Export started. Check Tasks in GEE.")